In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

class PolynomialRegression:
    def __init__(self, degree=2, method='gradient_descent', lr=0.001, iterations=10000):
        self.degree = degree
        self.method = method
        self.lr = lr
        self.iterations = iterations
        self.weights = None

    def create_polynomial_features(self, X):
        poly_features = [np.ones(X.shape[0])]  # Sabit terim (bias)
        for d in range(1, self.degree + 1):
            poly_features.append(X ** d)
        return np.column_stack(poly_features)

    def gradient_descent(self, X, y):
        theta = np.random.randn(X.shape[1])
        m = X.shape[0]

        for iteration in range(self.iterations):
            predictions = np.dot(X, theta)
            errors = predictions - y
            gradient = np.dot(X.T, errors) / m
            theta_new = theta - self.lr * gradient
            
            if np.linalg.norm(theta_new - theta) < 1e-6:
                print(f"Converged at iteration {iteration}")
                break
            
            theta = theta_new
        return theta
    
    def fit(self, X, y):
        X_poly = self.create_polynomial_features(X)
    
        if self.method == 'gradient_descent':
            self.weights = self.gradient_descent(X_poly, y)
        else:
            raise ValueError("'method' should be 'gradient_descent'")

    def predict(self, X):
        X_poly = self.create_polynomial_features(X)
        return np.dot(X_poly, self.weights)

# Verileri yükleme ve hazırlama
data = pd.read_csv('processed_dataset.csv')

# Kategorik değişkeni sayısal değere dönüştürme
le = LabelEncoder()
data['city'] = le.fit_transform(data['city'])

# Aykırı değerleri işleme
def handle_outliers_iqr_all_columns(data):
    for column_name in data.columns:
        Q1 = data[column_name].quantile(0.25)  # Alt çeyrek (25. yüzdelik)
        Q3 = data[column_name].quantile(0.75)  # Üst çeyrek (75. yüzdelik)
        IQR = Q3 - Q1  # IQR hesaplama

        lower_bound = Q1 - 1.5 * IQR  # Alt sınır
        upper_bound = Q3 + 1.5 * IQR  # Üst sınır

        # Alt sınırın altındaki ve üst sınırın üstündeki aykırı değerleri çeyrekler arası aralığa getirme
        data[column_name] = data[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return data

data = handle_outliers_iqr_all_columns(data)

# Özellik ve hedef değişkenleri ayırma
X = data.drop(columns=['opened_inside_year','carried_over_from','finished','carried_over_to']).values
y = data['opened_inside_year'].values

# Verileri normalleştirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Veriyi eğitim ve test setlerine ayırma
# İlk 81 satır 2021 yılına aittir
X_train, X_test, y_train, y_test = X_scaled[81:], X_scaled[:81], y[81:], y[:81]

# Gradient Descent ile model oluşturma ve eğitme
model_gd = PolynomialRegression(degree=2, method='gradient_descent', lr=0.1, iterations=1000)
model_gd.fit(X_train, y_train)

# Tahminler
y_pred_gd = model_gd.predict(X_test)

# Tahminleri ve gerçek y değerlerini yazdırma (Gradient Descent)
print("Gradient Descent Tahminleri:")
for i in range(len(y_test)):
    print("Tahmin Edilen:", y_pred_gd[i], "\tGerçek:", y_test[i])

# Her tahmin için mutlak yüzde hatasını hesaplama (gradient descent)
absolute_percentage_errors_gd = np.abs((y_test - y_pred_gd) / y_test)

# Ortalama mutlak yüzde hatasını hesaplama (gradient descent)
mape_gd = np.mean(absolute_percentage_errors_gd)

# MAPE'yi doğruluğa dönüştürme (gradient descent)
accuracy_gd = 1 - mape_gd

# Doğruluğu yüzdeye dönüştürme (gradient descent)
percentage_accuracy_gd = accuracy_gd * 100

print("Average Percentage Accuracy (Gradient Descent):", percentage_accuracy_gd)


Gradient Descent Tahminleri:
Tahmin Edilen: 100082.55374308366 	Gerçek: 103840.0
Tahmin Edilen: 64946.536743043354 	Gerçek: 67719.0
Tahmin Edilen: 25309.72023478658 	Gerçek: 25475.0
Tahmin Edilen: 17256.135293840533 	Gerçek: 18984.0
Tahmin Edilen: 71315.93027778751 	Gerçek: 73542.0
Tahmin Edilen: 31055.427482036175 	Gerçek: 33447.0
Tahmin Edilen: 100099.85349417878 	Gerçek: 103840.0
Tahmin Edilen: 87188.95376769919 	Gerçek: 87538.0
Tahmin Edilen: 55949.98587231991 	Gerçek: 59477.0
Tahmin Edilen: 72519.75319310382 	Gerçek: 83130.0
Tahmin Edilen: 76603.36488238204 	Gerçek: 76103.0
Tahmin Edilen: 34853.846432424136 	Gerçek: 38576.0
Tahmin Edilen: 24154.94917434027 	Gerçek: 25953.0
Tahmin Edilen: 21764.317271142372 	Gerçek: 23992.0
Tahmin Edilen: 99256.54816569624 	Gerçek: 103840.0
Tahmin Edilen: 57766.35917375332 	Gerçek: 62108.0
Tahmin Edilen: 12723.534785821805 	Gerçek: 16910.0
Tahmin Edilen: 98268.80932612762 	Gerçek: 86629.0
Tahmin Edilen: 49838.163708548076 	Gerçek: 51982.0
Tahmin Ed